# dev with dask

In [ ]:
from collections import defaultdict

import dask.array as da
import holoviews as hv
import hvplot
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import skimage
import tifffile as tff
from scipy import ndimage

from nima import io, nima, utils

%load_ext autoreload
%autoreload 2

fp = "../../tests/data/1b_c16_15.tif"

In [ ]:
daimg = da.from_zarr(tff.imread(fp, aszarr=True))
daimg

In [ ]:
utils.bg(daimg[0, 0].compute())

In [ ]:
def dabg(daimg):
    r = defaultdict(list)
    n_t, n_c = daimg.shape[:2]
    for t in range(n_t):
        for c in range(n_c):
            r[c].append(utils.bg(daimg[t, c].compute())[0])
    return pd.DataFrame(r)


dabg(daimg)

In [ ]:
def dabg_fg(daimg, erf_pvalue=1e-100, size=10):
    n_t, n_c = daimg.shape[:2]
    bgs = defaultdict(list)
    fgs = defaultdict(list)
    for t in range(n_t):
        p = np.ones(daimg.shape[-2:])
        multichannel = daimg[t].compute()
        for c in range(n_c):
            av, sd = utils.bg(multichannel[c])
            p *= utils.prob(multichannel[c], av, sd)
            bgs[c].append(av)
        mask = ndimage.median_filter((p) ** (1 / n_c), size=size) < erf_pvalue
        for c in range(n_c):
            fgs[c].append(np.ma.mean(np.ma.masked_array(multichannel[c], mask=~mask)))
    return pd.DataFrame(bgs), pd.DataFrame(fgs)


dfb, dff = dabg_fg(daimg)

In [ ]:
plt.subplot(121)
((dff - dfb)[0] / (dff - dfb)[2]).plot(marker="s")
plt.grid()
plt.subplot(122)
((dff - dfb)[2] / (dff - dfb)[1]).plot(marker="o")
plt.grid()

NEXT:
- make utils.bg and utils.prob work with dask arrays

In [ ]:
def dmask(daim, erf_pvalue=1e-100, size=10):
    n_c = daim.shape[0]
    im = daim[0].compute()
    p = utils.prob(im, *utils.bg(im))
    for c in range(1, n_c):
        im = daim[c].compute()
        p *= utils.prob(im, *utils.bg(im))
    p = ndimage.median_filter((p) ** (1 / n_c), size=size)
    mask = p < erf_pvalue
    return skimage.morphology.remove_small_objects(mask)
    # mask = skimage.morphology.remove_small_holes(mask)
    # return np.ma.masked_array(plane, mask=~mask), np.ma.masked_array(plane, mask=mask)


mask = dmask(daimg[2])

lab, nlab = ndimage.label(mask)
lab, nlab

In [ ]:
pr = skimage.measure.regionprops(lab, intensity_image=daimg[0][0])
pr[1].equivalent_diameter_area

In [ ]:
max_diameter = pr[0].equivalent_diameter_area
size = int(max_diameter * 0.3)
size

In [ ]:
t = 0
mask = dmask(daimg[t])
# skimage.io.imshow(mask)
lab, nlab = ndimage.label(mask)

distance = ndimage.distance_transform_edt(mask)
# distance = skimage.filters.gaussian(distance, sigma=0)   min_distance=size,
coords = skimage.feature.peak_local_max(
    distance, footprint=np.ones((size, size)), labels=lab
)
mm = np.zeros(distance.shape, dtype=bool)
mm[tuple(coords.T)] = True
# markers, _ = ndimage.label(mm)
markers = skimage.measure.label(mm)

labels = skimage.segmentation.watershed(-distance, markers, mask=mask)

_, (ax0, ax1, ax2) = plt.subplots(1, 3)
ax0.imshow(distance)
ax1.imshow(labels)
ax2.imshow(labels == 3)
coords

In [ ]:
masks = [dmask(daimg[t]) for t in range(4)]

In [ ]:
masks = np.stack(masks)
masks.shape

In [ ]:
tff.imshow(masks)

In [ ]:
distance = ndimage.distance_transform_edt(masks)

distance = skimage.filters.gaussian(distance, sigma=5)

In [ ]:
import impy

impy.array(distance).imshow()

In [ ]:
for t in range(4):
    coords = skimage.feature.peak_local_max(distance[t], footprint=np.ones((130, 130)))
    print(coords)

In [ ]:
co = np.stack([coords, coords, coords, coords])

In [ ]:
coords.T

In [ ]:
mm = np.zeros(masks[0].shape, dtype=bool)
mm[tuple(co.T)] = True
# markers, _ = ndimage.label(mm)
markers = skimage.measure.label(np.stack([mm, mm, mm, mm]))

labels = skimage.segmentation.watershed(-distance, markers, mask=masks)

_, (ax1, ax2) = plt.subplots(1, 2)
ax1.imshow(labels[3])
ax2.imshow(labels[3] == 4)

In [ ]:
img = tff.imread(fp)

In [ ]:
dim = io.read_image(fp, channels=["R", "G", "C"])

In [ ]:
bg_params = nima.BgParams()
res = nima.bg(dim, bg_params)
bgs = res[1]

In [ ]:
def ratio(t, roi):
    if not np.any(labels[t] == roi):
        return np.nan, np.nan
    g = img[t, 0][labels[t] == roi].mean() - bgs["G"][t]
    r = img[t, 1][labels[t] == roi].mean() - bgs["R"][t]
    c = img[t, 2][labels[t] == roi].mean() - bgs["C"][t]
    return g / c, c / r


ratio(1, 4)

In [ ]:
rph = defaultdict(list)
rcl = defaultdict(list)
for roi in range(1, 5):
    for t in range(4):
        ph, cl = ratio(t, roi)
        rph[roi].append(ph)
        rcl[roi].append(cl)

plt.plot(rph[1])
plt.plot(rph[2])
plt.plot(rph[3])
plt.plot(rph[4])

In [ ]:
plt.plot(rcl[1])
plt.plot(rcl[2])
plt.plot(rcl[3])
plt.plot(rcl[4])

In [ ]:
t = 2
mask = dmask(daimg[t])
# skimage.io.imshow(mask)
lab, nlab = ndimage.label(mask)
lab[~mask] = -1
# lab[lab==1] = -1
if np.any(lab == 0):
    labels_ws = skimage.segmentation.random_walker(
        daimg[t, 1].compute(), lab, beta=1e10, mode="bf"
    )
else:
    labels_ws = lab
# labels_ws = skimage.segmentation.random_walker(-distance, lab, beta=10000, mode="bf")

_, (ax1, ax2) = plt.subplots(1, 2)
ax1.imshow(labels_ws)
ax2.imshow(labels_ws == 2)

In [ ]:
imar = impy.imread(fp)

imar.label_threshold()

In [ ]:
imar[:, 2].imshow(label=1)

In [ ]:
def dmask0(im, erf_pvalue=1e-100, size=10):
    p = utils.prob(im[0], *utils.bg(im[0]))
    for img in im[1:]:
        p *= utils.prob(img, *utils.bg(img))
    p = ndimage.median_filter((p) ** (1 / len(im)), size=size)
    mask = p < erf_pvalue
    return skimage.morphology.remove_small_objects(mask)

In [ ]:
dmask0(imar[1])

In [ ]:
plt.imshow(skimage.measure.label(mask))

In [ ]:
distance = skimage.filters.gaussian(distance, sigma=30)
tff.imshow(distance)

In [ ]:
np.transpose(np.nonzero(skimage.morphology.local_maxima(distance)))

In [ ]:
tff.imshow(ndimage.label(mask)[0])

In [ ]:
res[1]

In [ ]:
res[2]["G"][2][0]

In [ ]:
res[1].plot()

In [ ]:
res[1].hvplot()

In [ ]:
xim = dim
xim.data

In [ ]:
xim.coords["Y"]

In [ ]:
xim.sel(C="G")[1, 0].hvplot(width=400, height=300)

In [ ]:
xim[0, :, 0].hvplot(
    width=300,
    subplots=True,
    by="C",
)

In [ ]:
hvplot.extension(
    "bokeh",
    "matplotlib",
)

In [ ]:
img = xim.sel(C="R")[0][0]

hvimg = hv.Image(img)

In [ ]:
hvimg

In [ ]:
# %%opts Image [aspect=1388/1038]

f = xim.sel(C="R")[:, 0].hvplot(
    frame_width=300,
    frame_height=200,
    subplots=True,
    col="T",
    yaxis=False,
    colorbar=False,
    xaxis=False,
    cmap="Reds",
) + xim.sel(C="C")[:, 0].hvplot(
    subplots=True, col="T", yaxis=False, colorbar=False, xaxis=False, cmap="Greens"
)
f

In [ ]:
import bioio

bioio.__version__

In [ ]:
hv.opts.defaults(
    hv.opts.Image(aspect=1388 / 1038),
    hv.opts.Image("Cyan", cmap=plt.cm.Blues),
    hv.opts.Image("Green", cmap=plt.cm.Greens),
    hv.opts.Image("Red", cmap=plt.cm.Reds),
)

In [ ]:
dim.sel(C="C", Z=0)

In [ ]:
chans = (
    hv.Image(dim.sel(C="C", Z=0)[0], group="cyan")
    + hv.Image(dim.sel(C="G", Z=0)[2], group="green")
    + hv.Image(dim.sel(C="R", Z=0)[1], group="red")
)

chans

In [ ]:
hv.save(chans, "a.png")

# Holoviews

In [ ]:
hv.extension("bokeh")
cm = plt.cm.inferno_r
channels = ["G", "R", "C"]
dim = io.read_image(fp, channels)

In [ ]:
dimm = nima.median(dim)
f = nima.plot_img(dimm, cmap=cm)

In [ ]:
f

In [ ]:
hv.opts.defaults(
    hv.opts.Image(aspect=512 / 512),
    hv.opts.Image("Cyan", cmap=plt.cm.Blues),
    hv.opts.Image("Green", cmap=plt.cm.Greens),
    hv.opts.Image("Red", cmap=plt.cm.Reds),
)

chans = (
    hv.Image(dim.sel(C="C", Z=0)[0], group="cyan")
    + hv.Image(dim.sel(C="G", Z=0)[0], group="green")
    + hv.Image(dim.sel(C="R", Z=0)[0], group="red")
)

chans

In [ ]:
nima.plot_img(dimm.sel(Z=0) - dim.sel(Z=0))

In [ ]:
c = [(i, hv.Image(im)) for i, im in enumerate(dim.sel(C="C", Z=0))]
c = hv.HoloMap(c, kdims=["Frame"])
g = [(i, hv.Image(im)) for i, im in enumerate(dim.sel(C="G", Z=0))]
g = hv.HoloMap(g, kdims=["Frame"])
r = [(i, hv.Image(im)) for i, im in enumerate(dim.sel(C="R", Z=0))]
r = hv.HoloMap(r, kdims=["Frame"])

In [ ]:
hv.output(holomap="auto")
hv.opts.defaults(hv.opts.Image(cmap="viridis"))
(c + g).select(Frame={0, 5, 6, 7, 10, 30}).cols(2)

In [ ]:
c[::20].overlay("Frame")

In [ ]:
wl = hv.Dimension("excitation wavelength", unit="nm")
c = c.add_dimension(wl, 1, 458)
g = g.add_dimension(wl, 1, 488)
r = r.add_dimension(wl, 1, 561)

channels = c.clone()
channels.update(g)
channels.update(r)

In [ ]:
hv.opts.defaults(hv.opts.Image(cmap="viridis", frame_width=300, aspect="equal"))
channels[::5].grid(["Frame", "excitation wavelength"])

In [ ]:
t = [(i, hv.Image(im)) for i, im in enumerate(dim.sel(C="C", Z=0))]

In [ ]:
hv.HoloMap(
    [(i, hv.Image(im)) for i, im in enumerate(dim.sel(C="C", Z=0))], kdims=["frame"]
)

In [ ]:
[str(k) for k in dim.C.data]

In [ ]:
hv.NdLayout(
    {
        k: hv.HoloMap(
            [(i, hv.Image(im)) for i, im in enumerate(dim.sel(C=k, Z=0))],
            kdims=["frame"],
        )
        for k in dim.C.data
    },
    kdims=["C"],
)[::4]

In [ ]:
hv.opts.defaults(hv.opts.Image(cmap="viridis"), hv.opts.Image("A", aspect=2))
im = hv.Image(dim.sel(Z=0, C="G")[1])
im2 = hv.Image(dim.sel(Z=0, C="C")[1])
im3 = hv.Image(dimm.sel(Z=0, C="C")[1])
(
    (im * hv.HLine(y=350 * 0.2))
    + im.sample(Y=350 * 0.2)
    + (im2 * hv.VLine(x=30))
    + im.sample(X=76) * im3.sample(X=30)
).cols(3)